In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as functional

import torchvision
import torchvision.transforms as transforms    

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.6.0
0.7.0


In [3]:
train_set=torchvision.datasets.FashionMNIST(
    root="./data",train=True,download=True,
    transform=transforms.Compose([transforms.ToTensor()]))

In [84]:
from itertools import product

In [85]:
parameters=dict(
lr=[0.01,0.001],
batch_size=[100,1000,10000],
shuffle=[True,False])

In [86]:
param_values=[v for v in parameters.values()]
param_values

[[0.01, 0.001], [100, 1000, 10000], [True, False]]

In [87]:
for lr,batch_size,shuffle in product(*param_values):
    print(lr,batch_size,shuffle)

0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.01 10000 True
0.01 10000 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False
0.001 10000 True
0.001 10000 False


In [1]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)
        
    def forward(self,t):
        # layer1-input layer
        t=t
        
        # layer2-hidden conv layer        
        t=self.conv1(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer3-hidden conv layer        
        t=self.conv2(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer4-hidden linear layer        
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=functional.relu(t)
        
        # layer5-hidden linear layer        
        t=self.fc2(t)
        t=functional.relu(t)
        
        # layer6-output layer        
        t=self.out(t)
        
        return t

NameError: name 'nn' is not defined

In [90]:
for lr, batch_size, shuffle in product(*param_values):
    network=Network()

    train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=shuffle)
    optimizer=optim.Adam(network.parameters(),lr=lr)

    images,labels=next(iter(train_loader))
    grid=torchvision.utils.make_grid(images)

    comment=f" batch_size={batch_size} lr={lr} shuffle={shuffle}"
    tb=SummaryWriter(comment=comment)
    tb.add_image("images",grid)
    tb.add_graph(network,images)

    for epoch in range(5):
        total_loss=0
        total_correct=0

        for batch in train_loader:   #get Batch
            images,labels=batch

            preds=network(images)  #passing the Batch Images
            loss=functional.cross_entropy(preds,labels)   #calculating Loss

            optimizer.zero_grad()  #Zero gradient
            loss.backward()  # Calculate Gradients
            optimizer.step()  #updating weights

            total_loss+=loss.item() * batch_size
            total_correct+=get_num_correct(preds,labels)

        tb.add_scalar("Loss",total_loss,epoch)
        tb.add_scalar("Number Correct",total_correct,epoch)
        tb.add_scalar("Accuracy",total_correct/len(train_set),epoch)

        for name,weight in network.named_parameters():
            tb.add_histogram(name,weight,epoch)
            tb.add_histogram(f"{name}.grad",weight.grad,epoch)

        print("epoch:",epoch,"\ttotal correct:",total_correct,"\tloss:",total_loss)

    tb.close()

epoch: 0 	total correct: 45445 	loss: 37789.141592383385
epoch: 1 	total correct: 50587 	loss: 25209.98557806015
epoch: 2 	total correct: 51641 	loss: 22463.973058760166
epoch: 3 	total correct: 51876 	loss: 21724.156667292118
epoch: 4 	total correct: 52154 	loss: 21208.325712382793
epoch: 0 	total correct: 44720 	loss: 39576.828956604004
epoch: 1 	total correct: 50530 	loss: 25688.51349800825
epoch: 2 	total correct: 51418 	loss: 23184.282341599464
epoch: 3 	total correct: 51952 	loss: 21709.263309836388
epoch: 4 	total correct: 52156 	loss: 21074.83551055193
epoch: 0 	total correct: 39947 	loss: 53183.35747718811
epoch: 1 	total correct: 48487 	loss: 30718.659669160843
epoch: 2 	total correct: 50415 	loss: 25774.30111169815
epoch: 3 	total correct: 51494 	loss: 23071.226447820663
epoch: 4 	total correct: 52302 	loss: 20894.53077316284
epoch: 0 	total correct: 37113 	loss: 57991.402208805084
epoch: 1 	total correct: 47005 	loss: 32657.3386490345
epoch: 2 	total correct: 49585 	loss: 2

In [2]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [6]:
class RunBuilder():
    
    @staticmethod
    def get_runs(params):
        Run=namedtuple("Run",params.keys())
        
        runs =[]
        for v in product(*params.values()):
            runs.append(Run(*v))
        
        return runs

In [7]:
params=OrderedDict(lr=[0.01,0.001],
                   batch_size=[100,1000])

In [8]:
runs=RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=100),
 Run(lr=0.01, batch_size=1000),
 Run(lr=0.001, batch_size=100),
 Run(lr=0.001, batch_size=1000)]

In [10]:
run=runs[0]
run

Run(lr=0.01, batch_size=100)

In [11]:
print(run.lr,run.batch_size)

0.01 100


In [12]:
for run in runs:
    print(run,run.lr,run.batch_size)

Run(lr=0.01, batch_size=100) 0.01 100
Run(lr=0.01, batch_size=1000) 0.01 1000
Run(lr=0.001, batch_size=100) 0.001 100
Run(lr=0.001, batch_size=1000) 0.001 1000
